# Imports

In [1]:
import os
import json
import yaml
import numpy as np
import pandas as pd

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

# Variables

In [2]:
path_pdf = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\cencora_earning_reports\AmerisourceBergen-Reports-Fiscal-2023-Second-Quarter-Results-2023.pdf"

embed_model= "text-embedding-ada-002"
vectore_store_persist_directory = 'vector_store'

# Config

In [23]:
import sys
import os
sys.path.append(os.path.expanduser("~/.utils"))

from util_openai import get_config_dict

config = get_config_dict(env=None)
config['openai_api_key'][:4]

'f400'

# Load pdf

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(path_pdf)
documents = loader.load()

# Split Documents Into Chunks

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Create Embeddings

# Create Vector Store and Retriever

In [6]:
# import glob
# glob.glob(str(os.path.expanduser("~/.certificates"))+"/*.crt")

In [7]:
# os.environ["REQUESTS_CA_BUNDLE"] = os.path.expanduser('~/.certificates/hugging_face.crt')
# os.environ["REQUESTS_CA_BUNDLE"] = os.path.expanduser('~/.certificates/openai_public.crt')

In [8]:
# openai
env = 'openai'


if env == 'openai':
    import openai

    print(f'Setting environment for: {env}\n\n')
    openai.api_key = config['openai_api_key']
    openai.api_base = config['openai_api_base']
    openai.api_type = 'azure'
    openai.api_version = config['openai_api_version']

    # set env keys
    keys = ["OPENAI_API_KEY","OPENAI_API_TYPE","OPENAI_API_BASE","OPENAI_API_VERSION"]
    for key in keys:
        os.environ[key] = config.get(key)

Setting environment for: openai




In [9]:
if 'OPENAI_API_BASE' in os.environ:
    del os.environ['OPENAI_API_BASE']

llm = AzureChatOpenAI(openai_api_type="azure",
                openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86",
                deployment_name='gpt-35-turbo-16k',
                model_name="text-davinci-002",
                azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev",
                openai_api_version="2023-03-15-preview"
                  )


C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [10]:
from langchain.embeddings import AzureOpenAIEmbeddings

embed_model= "text-embedding-ada-002"
model_kwargs = dict(azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev")
embeddings = AzureOpenAIEmbeddings(model=embed_model,
                              chunk_size=1,
                              openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86",
                              # model_kwargs=model_kwargs
                              openai_api_base="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev/"
                             )
embeddings

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_community\embeddings\azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev/ to https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev//openai.
  warnings.warn(


AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000021628C9F950>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000021628C8B510>, model='text-embedding-ada-002', deployment=None, openai_api_version='2023-05-15', openai_api_base='https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev//openai', openai_api_type='azure', openai_proxy='', embedding_ctx_length=8191, openai_api_key='242ad9b0c6d84b5fb4307061d5f05a86', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, azure_endpoint=None, azure_ad_token=None, azure_ad_token_provider=None, validate_base_url=True)

In [11]:
from langchain.vectorstores import FAISS

pages = loader.load_and_split()
vector_store = FAISS.from_documents(documents=pages, embedding=embeddings)
vector_store.save_local(vectore_store_persist_directory)

# LLM

In [12]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(**config['kw_azure_llm'],temperature=0.4)

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://qa.gai.cencora.com/aoai to https://qa.gai.cencora.com/aoai/openai.
  warnings.warn(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azur

# Get vector store from local file

In [13]:
from langchain.vectorstores import FAISS

vector_store = FAISS.load_local(vectore_store_persist_directory, embeddings, allow_dangerous_deserialization=True,)
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2},
    return_source_documents=True,
)

# Create RetrivalQA Chain

In [14]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = retriever
)

# Create Query And Get Response

In [15]:
query = 'What are the year to year gain in revenue?'
qa.run(query)

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The year-to-year gain in revenue is 7.6%.'

In [16]:
result = qa({"query": query})
print(result['result'])

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The year-to-year gain in revenue is 7.6%.


## Using: ConversationalRetrievalChain

In [17]:
from langchain.chains import ConversationalRetrievalChain

In [18]:
qa = ConversationalRetrievalChain.from_llm(llm,retriever)

In [19]:
chat_history = []
query = "Is there gain or loss of revenue in this quarter?"

result = qa({'question': query, 'chat_history': chat_history})
print(result['answer'])

Based on the information provided, there is a decrease in revenue in the International Healthcare Solutions segment. However, there is an increase in revenue in the U.S. Healthcare Solutions segment.


In [20]:
chat_history = [(query,result['answer'])]
query = "Is there gain in revenue for U.S. Healthcare Solutions this quarter?"

result = qa({'question': query, 'chat_history': chat_history})
print(result['answer'])

Yes, the U.S. Healthcare Solutions segment experienced an increase in revenue this quarter. The revenue for the segment in the current year was $112,930,035, while in the previous year it was $103,922,410, representing an 8.7% increase.


# Script

In [21]:
%%writefile a.py
# imports
import os
import json
import yaml
import numpy as np
import pandas as pd

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

# config
def get_config_dict():
    import os
    import yaml

    with open(os.path.expanduser('~/.config/config.yaml')) as fh:
        config = yaml.safe_load(fh)

    # openai
    keys = ["OPENAI_API_KEY","OPENAI_API_TYPE","OPENAI_API_BASE","OPENAI_API_VERSION"]
    for key in keys:
        os.environ[key] = config.get(key)

    return config

config = get_config_dict()

# variables
path_pdf = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\cencora_earning_reports\AmerisourceBergen-Reports-Fiscal-2023-Second-Quarter-Results-2023.pdf"

embed_model= "text-embedding-ada-002"
vectore_store_persist_directory = 'vector_store'

# load documents
loader = PyPDFLoader(path_pdf)
documents = loader.load()

# create embeddings
embed_model= "text-embedding-ada-002"
embeddings = AzureOpenAIEmbeddings(model=embed_model, chunk_size=1)

# create vector store
pages = loader.load_and_split()
vector_store = FAISS.from_documents(documents=pages, embedding=embeddings)
vector_store.save_local(vectore_store_persist_directory)

# llm
llm = AzureChatOpenAI(**config['kw_azure_llm'],temperature=0.4)

# load vector store from local file
vector_store = FAISS.load_local(vectore_store_persist_directory, embeddings)
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2},
    return_source_documents=True
)

# question answer
qa = ConversationalRetrievalChain.from_llm(llm, retriever)
chat_history = []
query = "Is there gain or loss of revenue in this quarter?"
result = qa({'question': query, 'chat_history': chat_history})
print(result['answer'])

# another chat
print()
chat_history = [(query,result['answer'])]
query = "Is there gain in revenue for U.S. Healthcare Solutions this quarter?"

result = qa({'question': query, 'chat_history': chat_history})
print(result['answer'])

Overwriting a.py


In [22]:
import pyperclip

pyperclip.copy(os.getcwd())